In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from helpers2 import S3Connection

In [ ]:
from postal.expand import expand_address

Import des données

In [3]:
s3 = S3Connection(bucket_name="clichere/diffusion")

Connection successful


In [9]:
path_logements_existants = "DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
df = s3.read_csv_from_s3(path_logements_existants)

In [ ]:
path_vf_2022 = "valeursfoncieres/vf_2022.csv"
vf2022 = s3.read_csv_from_s3(path_vf_2022)

path_vf_2021 = "valeursfoncieres/vf_2021.csv"
vf2021 = s3.read_csv_from_s3(path_vf_2021)

# FONCTIONS UTILES

Les deux fonctions suivantes permettent de normaliser les adresses présentes dans les deux bases de données. 

In [ ]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return normalized[0] if normalized else None  # Ne garde que la première version de la normalisation
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

Cette fonction est différente de la précédente car le problème de l'arrondissement après Paris ne se trouvait que dans la base de données vf. 

In [ ]:
import re

def normalize_vf_address(address, code_postal):
    """ Normalise l'adresse et supprime l'arrondissement après 'PARIS' uniquement si le département est 75. """
    normalized_address = normalize_address(address)  

    if pd.notna(normalized_address) and str(code_postal).startswith("75"):
        # Supprime le numéro après "PARIS" uniquement pour le département 75
        normalized_address = re.sub(r'(paris) \d{2}$', r'\1', normalized_address, flags=re.IGNORECASE)
    
    return normalized_address

Le code suivant permet de relier les deux bases de données à partir des adresses normalisées en établissant un seuil d'écart de 5% entre les surfaces de chaque base (surface habitable / surface loi carrez).

In [ ]:
def test_match2(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code departement']==department].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code departement']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        #On distingue les adresses uniques et les adresses en doublons pour avoir deux catégories d'adresses et trouver celles communes aux deux bases
        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        # On regarde les adresses communes
        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            # On crée des sous dataframe contenant les lignes avec les mêmes adresses
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            # Boucle sur les rangs des sous dataframe
            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 5% et parmi les lignes avec des surfaces inf au seuil si jamais la valeur foncière est supérieure à celle de la ligne d'avant on la conserve pour avoir la plus grande
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df


Même fonction mais pour seuil de 10%. 

In [ ]:
def test_match3(vf, df):
    merged = []

    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code departement']==department].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code departement']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 10%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.1:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df

Même fonction mais avec seuil de 3%

In [ ]:
def test_match4(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code departement']==department].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code departement']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 3%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.03:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df

Fonction pour compiler deux dataframe. 

In [ ]:
# Compiler les deux DataFrames
def compiler_dataframes(df1, df2):
    #concatène les deux DataFrames
    df_compilé = pd.concat([df1, df2], ignore_index=True)
    #supprime doublons
    df_compilé = df_compilé.drop_duplicates()
    return df_compilé

# MATCHING ANNEE 2022

Normalisation des adresses dans le fichier vf.

In [ ]:
# Convertir explicitement toutes les colonnes en chaînes
vf2022['Adresse'] = vf2022['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf2022['Type de voie'].fillna('').astype(str) + " " + \
                vf2022['Voie'].fillna('').astype(str) + ", " + \
                vf2022['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf2022['Commune'].fillna('').astype(str)

# On crée la colonne Adresse

vf2022['Adresse'] = vf2022['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

On sélectionne les données de 2022.

In [12]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
df2022 = df[df['Date_réception_DPE'].dt.year == 2022].copy()

On enlève les valeurs manquantes et on convertit le format du numéro de département.

In [ ]:
vf2022 = vf2022[vf['Code departement'].notna()]
df2022 = df2022[df2022['N°_département_(BAN)'].notna()]

In [ ]:
vf2022['Code departement'] = vf2022['Code departement'].astype(str)
df2022.loc[:, 'N°_département_(BAN)'] = df2022['N°_département_(BAN)'].astype(str)

## TEST AVEC 5% POUR LES SURFACES

In [ ]:
#application fonction
test2 = test_match2(vf2022, df2022)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

Quelques lignes sont en doublons dans le résultat final, nous pouvons les enlever. 

In [93]:
result_df = test2.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [95]:
result_df.to_csv("result5%.csv", index=False)

In [94]:
print(len(result_df))

77673


Nous obtenons un dataframe de 77673 lignes pour les logements existants avec un seuil de 5%.

## TEST AVEC 10% POUR LES SURFACES

In [ ]:
#application fonction
test3 = test_match3(vf2022, df2022)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

Même raisonnement que précédemment on enlève les doublons. 

In [122]:
result10 = test3.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [123]:
print(len(result10))

89539


In [124]:
result10.to_csv("result10%.csv", index=False)

On obtient un dataframe plus important avec 89 539 lignes. 

## TEST SEUIL DE 3%

In [ ]:
#application fonction
test4 = test_match4(vf2022, df2022)

In [127]:
result3 = test4.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [128]:
print(len(result3))

69752


In [129]:
result3.to_csv("result3%.csv", index=False)

Nous allons seulement pouvoir compiler les données pour les années : 2021 et 2022 car nous n'avons pas les données valeurs foncières de 2023. 

# 2021

On utilise le même raisonnement que pour 2022 on formate les données. 

On crée la colonne adresse dans le dataframe vf pour que l'adresse soit utilisable. 

In [6]:
# Convertir explicitement toutes les colonnes en chaînes
vf2021['Adresse'] = vf2021['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf2021['Type de voie'].fillna('').astype(str) + " " + \
                vf2021['Voie'].fillna('').astype(str) + ", " + \
                vf2021['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf2021['Commune'].fillna('').astype(str)

vf2021['Adresse'] = vf2021['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

On ne garde que les données de 2021. 

In [7]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
df2021 = df[df['Date_réception_DPE'].dt.year == 2021].copy()

On enlève les valeurs manquantes et on convertit le format du numéro de département. 

In [8]:
vf2021 = vf2021[vf2021['Code departement'].notna()]
df2021 = df2021[df2021['N°_département_(BAN)'].notna()]

In [9]:
vf2021['Code departement'] = vf2021['Code departement'].astype(str)
df2021.loc[:, 'N°_département_(BAN)'] = df2021['N°_département_(BAN)'].astype(str)

MATCHING 5%

In [15]:
dfexistant2021 = test_match2(vf2021, df2021)

Traitement du département : 25
Traitement du département : 44
Traitement du département : 36
Traitement du département : 29
Traitement du département : 43
Traitement du département : 06
Traitement du département : 88
Traitement du département : 78
Traitement du département : 14
Traitement du département : 80
Traitement du département : 54
Traitement du département : 82
Traitement du département : 49
Traitement du département : 86
Traitement du département : 52
Traitement du département : 64
Traitement du département : 59
Traitement du département : 77
Traitement du département : 72
Traitement du département : 67
Traitement du département : 60
Traitement du département : 89
Traitement du département : 41
Traitement du département : 30
Traitement du département : 69
Traitement du département : 81
Traitement du département : 85
Traitement du département : 57
Traitement du département : 2B
Traitement du département : 76
Traitement du département : 35
Traitement du département : 56
Traiteme

In [16]:
dfexistant2021 = dfexistant2021.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [18]:
print(len(dfexistant2021))

21411


On obtient un fichier final pour le matching à 5% avec 21 411 lignes. 

# DATA FRAME FINAL POUR 5%

On récupère les dataframe à 5% pour les années 2021 et 2022. 

In [ ]:
df1 = dfexistant2021
df2 = pd.read_csv("result5%.csv")

/var/folders/bf/ymcp1z4s7n77n5bhz3q_hgqc0000gn/T/ipykernel_1211/1554524822.py:1: DtypeWarning: Columns (14,15,30,52,58,60,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("result5%_2021_compilé.csv")
/var/folders/bf/ymcp1z4s7n77n5bhz3q_hgqc0000gn/T/ipykernel_1211/1554524822.py:2: DtypeWarning: Columns (14,15,30,52,58,60,62,65,67,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("result5%_2022_compilé.csv")


In [175]:
dffinal5 = compiler_dataframes(df1, df2)

On utilise la fonction compiler et on exporte le fichier au format csv. 

In [ ]:
dffinal5.to_csv("result5%_surfacecarrez.csv", index=False)

## TEST AVEC SURFACE REELLE BATIMENT

Jusqu'à présent nous avions utilisé la colonne surface loi carrez pour le matching, nous pouvons essayer de considérer la surface réelle du bâtiment. Il faut donc modifier les codes utilisés. 

# 2022

Même code mais changement de colonne. 

In [18]:
def test_matchbati(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code departement']==department].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code departement']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface reelle bati'] = pd.to_numeric(vf_dept['Surface reelle bati'], errors='coerce')
        vf_dept = vf_dept[vf_dept['Surface reelle bati'].notna() & (vf_dept['Surface reelle bati'] > 0)]
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface reelle bati']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 5%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df


In [ ]:
testbati = test_matchbati(vf2022, df2022)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

In [22]:
resultsurfacebati5 = testbati.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [23]:
print(len(resultsurfacebati5))

142926


On obtient un dataframe plus grand avec 142 926 lignes. 

# 2021

In [39]:
testbati2021 = test_matchbati(vf2021, df2021)

Traitement du département : 25
Traitement du département : 44
Traitement du département : 36
Traitement du département : 29
Traitement du département : 43
Traitement du département : 06
Traitement du département : 88
Traitement du département : 78
Traitement du département : 14
Traitement du département : 80
Traitement du département : 54
Traitement du département : 82
Traitement du département : 49
Traitement du département : 86
Traitement du département : 52
Traitement du département : 64
Traitement du département : 59
Traitement du département : 77
Traitement du département : 72
Traitement du département : 67
Traitement du département : 60
Traitement du département : 89
Traitement du département : 41
Traitement du département : 30
Traitement du département : 69
Traitement du département : 81
Traitement du département : 85
Traitement du département : 57
Traitement du département : 2B
Traitement du département : 76
Traitement du département : 35
Traitement du département : 56
Traiteme

In [41]:
result2021surfacebati5 = testbati2021.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [42]:
print(len(result2021surfacebati5))

37772


On obtient encore une fois un dataframe plus grand de 37 772 lignes. 

## COMPILER LES DEUX 

On compile les deux fichiers obtenus pour le matching à 5% avec la surface réelle du batiment. 

In [44]:
resultsurfacereelle5 = compiler_dataframes(resultsurfacebati5, result2021surfacebati5)

In [45]:
print(len(resultsurfacereelle5))

180698


In [46]:
result2021surfacebati5.to_csv("result5%_surfacereellebati.csv", index=False)

On obtient un fichier plus grand que celui obtenu avec la surface loi carrez. 